<a href="https://colab.research.google.com/github/orleansb/streamlit-example/blob/master/MidsemProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.1 MB/s eta 0:00:00


Importing all the necessary libraries

In [ ]:
import pandas as pd
import os
import sklearn
import numpy as np
import pandas as pd
import numpy as np, pandas as pd
from sklearn import tree, metrics
from sklearn.preprocessing import  LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, KFold
import streamlit as st
import pickle
import numpy as np


Reading the players 21 file and saving it in data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/players_21.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 15.9+ MB


Picking out the columns with categorical values

In [ ]:
categorical_columns = data.select_dtypes(include=['object', 'category']).columns

In [ ]:
fifa = pd.DataFrame(data)

In [ ]:
player_names = fifa['long_name']

## Encoding the categorical data using Label Encoder

In [ ]:
selected_columns_df = fifa[categorical_columns]
label_encoder = LabelEncoder()

In [ ]:
label_encoded = selected_columns_df.apply(lambda x: label_encoder.fit_transform(x))
categorical_values = pd.DataFrame(label_encoded)

In [ ]:
fifa= fifa.drop(selected_columns_df ,axis=1)
fifa

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,158023,93,93,103500000.0,560000.0,33,170,72,241.0,1.0,...,96,32,35,24,6,11,15,14,8,NaN
1,20801,92,92,63000000.0,220000.0,35,187,83,45.0,1.0,...,95,28,32,24,7,11,15,14,11,NaN
2,188545,91,91,111000000.0,240000.0,31,184,80,21.0,1.0,...,88,35,42,19,15,6,12,8,10,NaN
3,190871,91,91,132000000.0,270000.0,28,175,68,73.0,1.0,...,93,35,30,29,9,9,15,15,11,NaN
4,192985,91,91,129000000.0,370000.0,29,181,70,10.0,1.0,...,91,68,65,53,15,13,5,10,13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,257710,47,52,70000.0,1000.0,21,177,70,112165.0,1.0,...,40,45,56,47,12,13,8,14,6,NaN
18940,257933,47,53,70000.0,1000.0,21,174,68,112540.0,1.0,...,35,43,42,53,8,8,13,14,10,NaN
18941,257936,47,47,45000.0,2000.0,28,185,79,111774.0,1.0,...,35,38,43,45,8,5,11,5,7,NaN
18942,258736,47,67,130000.0,500.0,17,171,58,1920.0,4.0,...,45,18,11,13,11,13,9,9,6,NaN


Adding the encoded data samples back to the main dataset


In [ ]:
fifa = pd.concat([fifa,categorical_values], axis=1)
fifa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(94)
memory usage: 15.9 MB


## Feature Engineering

In [ ]:
corr_matrix = fifa.corr()
low_corr_columns = corr_matrix.index[corr_matrix['overall'] < 0.6]
fifa = fifa.drop(columns=low_corr_columns)

Finding the samples with the maximum correlation and dropping the samples that fall beneath a certain threshold


In [ ]:
corr_matrix = fifa.corr()
corr_matrix["overall"].sort_values(ascending=False)
fifa
'overall', 'potential', 'realease_clause_eur',

,overall,potential,release_clause_eur,passing,dribbling,movement_reactions,mentality_composure,lf,cf,rf,lam,cam,ram,lm,lcm,cm,rcm,rm
0,93,93,138400000.0,91.0,95.0,94,96,132,132,132,266,266,266,249,216,216,216,249
1,92,92,75900000.0,81.0,89.0,95,95,131,131,131,263,263,263,246,197,197,197,246
2,91,91,132000000.0,78.0,85.0,93,88,128,128,128,254,254,254,233,187,187,187,233
3,91,91,166500000.0,86.0,94.0,91,93,130,130,130,265,265,265,248,206,206,206,248
4,91,91,161000000.0,93.0,88.0,91,91,129,129,129,264,264,264,247,217,217,217,247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,47,52,57000.0,26.0,27.0,48,40,15,15,15,15,15,15,19,21,21,21,19
18940,47,53,72000.0,49.0,47.0,50,35,29,29,29,46,46,46,44,49,49,49,44
18941,47,47,47000.0,49.0,46.0,44,35,31,31,31,48,48,48,52,48,48,48,52
18942,47,67,165000.0,40.0,53.0,53,45,34,34,34,53,53,53,48,42,42,42,48


## Imputing the missing values

In [ ]:
fifa=fifa.fillna(method='ffill')
fifa=fifa.fillna(method='bfill')

Setting the

In [ ]:
Y = fifa['overall']
X =fifa.drop("overall",axis=1)

In [ ]:
X.head()

,potential,release_clause_eur,passing,dribbling,movement_reactions,mentality_composure,lf,cf,rf,lam,cam,ram,lm,lcm,cm,rcm,rm
0,93,138400000.0,91.0,95.0,94,96,132,132,132,266,266,266,249,216,216,216,249
1,92,75900000.0,81.0,89.0,95,95,131,131,131,263,263,263,246,197,197,197,246
2,91,132000000.0,78.0,85.0,93,88,128,128,128,254,254,254,233,187,187,187,233
3,91,166500000.0,86.0,94.0,91,93,130,130,130,265,265,265,248,206,206,206,248
4,91,161000000.0,93.0,88.0,91,91,129,129,129,264,264,264,247,217,217,217,247


In [ ]:
from sklearn.preprocessing import StandardScaler

## Scaling the data

In [ ]:
x=StandardScaler()

In [ ]:
scaled = x.fit_transform(X)
fifa = pd.DataFrame(scaled, columns=fifa.columns.drop('overall'))
X = fifa

In [ ]:
X

,potential,release_clause_eur,passing,dribbling,movement_reactions,mentality_composure,lf,cf,rf,lam,cam,ram,lm,lcm,cm,rcm,rm
0,3.586563,13.461596,3.281335,3.235687,3.554438,3.137573,3.327860,3.327860,3.327860,3.221565,3.221565,3.221565,3.032407,3.312891,3.312891,3.312891,3.032407
1,3.422893,7.187205,2.316107,2.642080,3.664174,3.055051,3.287564,3.287564,3.287564,3.165377,3.165377,3.165377,2.974117,2.846936,2.846936,2.846936,2.974117
2,3.259222,12.819098,2.026539,2.246342,3.444701,2.477402,3.166675,3.166675,3.166675,2.996812,2.996812,2.996812,2.721528,2.601696,2.601696,2.601696,2.721528
3,3.259222,16.282562,2.798721,3.136753,3.225227,2.890009,3.247268,3.247268,3.247268,3.202836,3.202836,3.202836,3.012977,3.067652,3.067652,3.067652,3.012977
4,3.259222,15.730416,3.474380,2.543146,3.225227,2.724966,3.206972,3.206972,3.206972,3.184106,3.184106,3.184106,2.993547,3.337415,3.337415,3.337415,2.993547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,-3.123941,-0.426693,-2.992643,-3.491860,-1.493455,-1.483625,-1.386794,-1.386794,-1.386794,-1.479521,-1.479521,-1.479521,-1.436476,-1.469281,-1.469281,-1.469281,-1.436476
18940,-2.960270,-0.425187,-0.772620,-1.513170,-1.273981,-1.896232,-0.822647,-0.822647,-0.822647,-0.898909,-0.898909,-0.898909,-0.950728,-0.782610,-0.782610,-0.782610,-0.950728
18941,-3.942295,-0.427697,-0.772620,-1.612104,-1.932402,-1.896232,-0.742055,-0.742055,-0.742055,-0.861450,-0.861450,-0.861450,-0.795289,-0.807134,-0.807134,-0.807134,-0.795289
18942,-0.668878,-0.415851,-1.641325,-0.919563,-0.944771,-1.071018,-0.621166,-0.621166,-0.621166,-0.767803,-0.767803,-0.767803,-0.873008,-0.954278,-0.954278,-0.954278,-0.873008


In [ ]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y,test_size=0.2,random_state=42)

## Training a model using XGBoost

In [ ]:
xg = xgb.XGBRegressor(n_jobs=-1, eval_metric='mae')
xg.fit(Xtrain, Ytrain)
xg_pred = xg.predict(Xtest)

In [ ]:
mse = mean_absolute_error(xg_pred,Ytest)
mse

0.9589931038005524

# Training a model using Random Forest

In [ ]:
rf=RandomForestRegressor()

In [ ]:
rf= RandomForestRegressor(n_estimators=90, random_state=42)

In [ ]:
rf.fit(Xtrain, Ytrain)

RandomForestRegressor(n_estimators=90, random_state=42)

In [ ]:
rf_pred = rf.predict(Xtest)

## Using cross validation for the Random Forest Model

In [ ]:
cv = KFold(n_splits=3)
PARAMETERS = {
    "max_depth": [2, 5, 6, 12],
    "n_estimators": [100, 500, 1000]
}
model_gs = GridSearchCV(rf, param_grid=PARAMETERS, cv=cv, scoring="neg_mean_absolute_error")
model_gs.fit(Xtrain, Ytrain)
y_pred = model_gs.best_estimator_.predict(Xtest)
y_pred

array([60.59352986, 62.2913202 , 58.08646243, ..., 73.54759657,
       62.63190754, 61.70929458])

In [ ]:
mae = mean_absolute_error(rf_pred,Ytest)

In [ ]:
print("Mean absolute error :" ,mae)

Mean absolute error : 0.9043341837482771


## Training a model using Gradient Boosting

In [ ]:
gbr = GradientBoostingRegressor()

In [ ]:
gbr = GradientBoostingRegressor(n_estimators=100,learning_rate=0.05,random_state=50,max_features=5)

In [ ]:
gbr.fit(Xtrain, Ytrain)

GradientBoostingRegressor(learning_rate=0.05, max_features=5, random_state=50)

In [ ]:
y_prediction = gbr.predict(Xtest)

In [ ]:
y_prediction

array([62.11373499, 62.46805594, 58.26973894, ..., 73.77860445,
       61.50509446, 62.15521865])

In [ ]:
mse_2 = mean_absolute_error(y_prediction,Ytest)

In [ ]:
print("Mean absolute error :" ,mse_2)

Mean absolute error : 1.3706478088122944


## Loading and pre-processing the test data



In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/archive/players_22.csv')

<ipython-input-321-ae9973701cdc>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv('/content/drive/MyDrive/archive/players_22.csv')


In [ ]:
categorical_columns = test_data.select_dtypes(include=['object', 'category']).columns
column_names = categorical_columns
column_names

Index(['player_url', 'short_name', 'long_name', 'player_positions', 'dob',
       'club_name', 'league_name', 'club_position', 'club_loaned_from',
       'club_joined', 'nationality_name', 'nation_position', 'preferred_foot',
       'work_rate', 'body_type', 'real_face', 'player_tags', 'player_traits',
       'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
       'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb',
       'lcb', 'cb', 'rcb', 'rb', 'gk', 'player_face_url', 'club_logo_url',
       'club_flag_url', 'nation_logo_url', 'nation_flag_url'],
      dtype='object')

In [ ]:
fifa_test = pd.DataFrame(test_data)

## Imputing the missing values

In [ ]:
fifa_test=fifa_test.fillna(method='ffill')
fifa_test=fifa_test.fillna(method='bfill')

In [ ]:
Y_1 = fifa_test['overall']

## Encoding the data using Label encoding

In [ ]:
selected_columns_df = fifa_test[column_names]
label_encoder = LabelEncoder()
label_encoded = selected_columns_df.apply(lambda x: label_encoder.fit_transform(x))
label_encoded
categorical_values = pd.DataFrame(label_encoded)

In [ ]:
fifa_test= fifa_test.drop(selected_columns_df ,axis=1)
fifa_test

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,158023,93,93,78000000.0,320000.0,34,170,72,73.0,1.0,...,96,20,35,24,6,11,15,14,8,50.0
1,188545,92,92,119500000.0,270000.0,32,185,81,21.0,1.0,...,88,35,42,19,15,6,12,8,10,50.0
2,20801,91,91,45000000.0,270000.0,36,187,83,11.0,1.0,...,95,24,32,24,7,11,15,14,11,50.0
3,190871,91,91,129000000.0,270000.0,29,175,68,73.0,1.0,...,93,35,32,29,9,9,15,15,11,50.0
4,192985,91,91,125500000.0,350000.0,30,181,70,10.0,1.0,...,89,68,65,53,15,13,5,10,13,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962,47,52,70000.0,1000.0,22,180,64,112541.0,1.0,...,37,38,43,48,6,10,5,15,13,21.0
19235,262040,47,59,110000.0,500.0,19,175,70,445.0,1.0,...,47,37,44,47,11,12,6,8,10,21.0
19236,262760,47,55,100000.0,500.0,21,178,72,111131.0,1.0,...,36,38,44,48,8,6,7,10,6,21.0
19237,262820,47,60,110000.0,500.0,19,173,66,111131.0,1.0,...,47,10,14,11,7,10,7,14,15,21.0


In [ ]:
fifa_test = pd.concat([fifa_test,categorical_values], axis=1)
fifa_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(94)
memory usage: 16.1 MB


## Creating a feature subset based on their correlation

In [ ]:
corr_matrix = fifa_test.corr()
low_corr_columns = corr_matrix.index[corr_matrix['overall'] < 0.6]
fifa_test = fifa_test.drop(columns=low_corr_columns)
fifa_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   overall              19239 non-null  int64  
 1   potential            19239 non-null  int64  
 2   wage_eur             19239 non-null  float64
 3   passing              19239 non-null  float64
 4   dribbling            19239 non-null  float64
 5   movement_reactions   19239 non-null  int64  
 6   mentality_composure  19239 non-null  int64  
 7   lf                   19239 non-null  int64  
 8   cf                   19239 non-null  int64  
 9   rf                   19239 non-null  int64  
 10  lam                  19239 non-null  int64  
 11  cam                  19239 non-null  int64  
 12  ram                  19239 non-null  int64  
 13  lm                   19239 non-null  int64  
 14  lcm                  19239 non-null  int64  
 15  cm                   19239 non-null 

,overall,potential,wage_eur,passing,dribbling,movement_reactions,mentality_composure,lf,cf,rf,lam,cam,ram,lm,lcm,cm,rcm,rm
0,93,93,320000.0,91.0,95.0,94,96,131,131,131,251,251,251,234,210,210,210,234
1,92,92,270000.0,79.0,86.0,93,88,128,128,128,245,245,245,224,186,186,186,224
2,91,91,270000.0,80.0,88.0,94,95,129,129,129,245,245,245,230,177,177,177,230
3,91,91,270000.0,86.0,94.0,89,93,128,128,128,249,249,249,232,195,195,195,232
4,91,91,350000.0,93.0,88.0,91,89,127,127,127,249,249,249,232,211,211,211,232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47,52,1000.0,46.0,48.0,53,37,30,30,30,47,47,47,46,48,48,48,46
19235,47,59,500.0,50.0,46.0,49,47,31,31,31,48,48,48,49,49,49,49,49
19236,47,55,500.0,45.0,49.0,46,36,31,31,31,48,48,48,47,49,49,49,47
19237,47,60,500.0,36.0,48.0,48,47,32,32,32,46,46,46,43,35,35,35,43


In [ ]:
X_1 =fifa_test.drop("overall",axis=1)

In [ ]:
corr_matrix["overall"].sort_values(ascending=False)
fifa_test

,overall,potential,wage_eur,passing,dribbling,movement_reactions,mentality_composure,lf,cf,rf,lam,cam,ram,lm,lcm,cm,rcm,rm
0,93,93,320000.0,91.0,95.0,94,96,131,131,131,251,251,251,234,210,210,210,234
1,92,92,270000.0,79.0,86.0,93,88,128,128,128,245,245,245,224,186,186,186,224
2,91,91,270000.0,80.0,88.0,94,95,129,129,129,245,245,245,230,177,177,177,230
3,91,91,270000.0,86.0,94.0,89,93,128,128,128,249,249,249,232,195,195,195,232
4,91,91,350000.0,93.0,88.0,91,89,127,127,127,249,249,249,232,211,211,211,232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47,52,1000.0,46.0,48.0,53,37,30,30,30,47,47,47,46,48,48,48,46
19235,47,59,500.0,50.0,46.0,49,47,31,31,31,48,48,48,49,49,49,49,49
19236,47,55,500.0,45.0,49.0,46,36,31,31,31,48,48,48,47,49,49,49,47
19237,47,60,500.0,36.0,48.0,48,47,32,32,32,46,46,46,43,35,35,35,43


#Scaling the data

In [ ]:
sc=StandardScaler()

In [ ]:
scaled1 = sc.fit_transform(X_1)
fifa_test = pd.DataFrame(scaled1, columns=fifa_test.columns.drop('overall'))
X_1 = fifa_test

In [ ]:
X_1train,X_1test,Y_1train,Y_1test=train_test_split(X_1,Y_1,test_size=0.2,random_state=42)

## Using the gradient boost regressor model to predict the players overall rating

In [ ]:
y1_prediction = gbr.predict(X_1test)

In [ ]:
y1_prediction

array([63.93741945, 59.38782353, 51.57115125, ..., 62.95270399,
       61.77879888, 66.3852088 ])

In [ ]:
mae_3 = mean_absolute_error(y1_prediction,Y_1test)

In [ ]:
print("Mean absolute error :" ,mae_3)

Mean absolute error : 1.4983550677362694


## Using the XG Regressor model to predict the players overall rating

In [ ]:
xg = xgb.XGBRegressor(learning_rate = 0.1,max_depth = 5,colsample_bytree = 0.5)
xg.fit(X_1train, Y_1train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
xg_predict = xg.predict( X_1test)
xg_predict

array([63.58793 , 58.307415, 51.5264  , ..., 62.353275, 62.122658,
       66.893135], dtype=float32)

In [ ]:
mse = mean_absolute_error(xg_pred,Ytest)
mse

0.9589931038005524

## Using the random forest model to predict the players overall rating

In [ ]:
y_pred = rf.predict(X_1test)

In [ ]:
y_pred

array([63.375, 59.15 , 52.575, ..., 62.325, 62.25 , 65.925])

In [ ]:
mae = mean_absolute_error(y_pred, Y_1test)

In [ ]:
mae

1.3801910083160083

In [ ]:
import pickle

In [ ]:
pickle.dump(rf,open('rf.pkl','wb'))

In [ ]:
with open('rf.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

In [ ]:
st.title("Player Rating Predictor")

DeltaGenerator()

In [ ]:
feature1 = st.text_input('Feature 1', value=0.0)
def predict_ratings (columns):
  columns = [float(feature1)]
  prediction = model.predict([feature1])
  return prediction





In [ ]:
my_col = []
predict_ratings()

In [ ]:
overall= st.number_input('overall', min_value=0.0, step=0.01, key='overall')
potential= st.number_input('potential', min_value=0.0, step=0.01, key='potential')
wage_eur = st.number_input('wage_eur', min_value=0.0, step=0.01, key='wage_eur')
passing = st.number_input('passing', min_value=0.0, step=0.01, key='passing')
dribbling = st.number_input('dribbling', min_value=0.0, step=0.01, key='dribbling')
movement_reactions = st.number_input('movement_reactions', min_value=0.0, step=0.01, key='movement_reactions')
mentality_composure = st.number_input('mentality_composure', min_value=0.0, step=0.01, key='mentality_composure')
lf = st.number_input('lf', min_value=0.0, step=0.01, key='lf')
cf = st.number_input('cf', min_value=0.0, step=0.01, key='cf')
rf = st.number_input('rf', min_value=0.0, step=0.01, key='rf')
lam = st.number_input('lam', min_value=0.0, step=0.01, key='lam')
cam = st.number_input('cam', min_value=0.0, step=0.01, key='cam')
ram = st.number_input('ram', min_value=0.0, step=0.01, key='ram')
lm = st.number_input('lm', min_value=0.0, step=0.01, key='lm')
lcm = st.number_input('lcm', min_value=0.0, step=0.01, key='lcm')
cm = st.number_input('cm', min_value=0.0, step=0.01, key='cm')
rcm = st.number_input('rcm', min_value=0.0, step=0.01, key='rcm')
rm = st.number_input('rm', min_value=0.0, step=0.01, key='rm')


In [ ]:
if st.button("Predict"):
    input_data = np.array([[overall, potential, wage_eur, passing, dribbling, movement_reactions, mentality_composure, lf, cf, rf, lam, cam, ram, lm, lcm, cm, rcm, rm]])

    prediction = model.predict(input_data)[0]
    confidence_score = model.predict_proba(input_data).max() * 100

    st.write(f"Player Rating Prediction: {prediction:.2f}")
    st.write(f"Confidence Score: {confidence_score:.2f}%")